In [1]:
from backbone.vgg16 import VGG16
from backbone.alexnet import AlexNet
from backbone.resnet import ResNet50, ResNet101, ResNet152
from backbone.inceptionv1 import InceptionV1
from object_detection.yolov1 import YOLOV1
import tensorflow as tf
from tensorflow import keras
from utiles.cifar10 import Cifar10
from utiles.directory_generator import ClassGenerator
from configs.yolov1_voc_config import YoloV1_config
from utiles.pascal import Pascal_VOC
from utiles.pascal_tfrecord import Pascal_VOC_TFRecord
from object_detection.yolov1_loss import YoloV1Loss
from callbacks.checkpoint_callback import checkpoint_callback as cck
from callbacks.lr_schedule import LearningRateScheduler
import os

In [2]:
import tensorflow as tf
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca.learn.tf2 import Estimator
import matplotlib.pyplot as plt
import numpy as np

Prepending /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
Adding /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/zoo/share/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.9.0-SNAPSHOT-jar-with-dependencies.jar to BIGDL_JARS
Prepending /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path


/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/bigdl/util/engine.py:41: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /usr/local/spark2, and pyspark is found in: /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
  warnings.warn(warning_msg)
/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/zoo/util/engine.py:42: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /usr/local/spark2, and pyspark is found in: /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/pyspark/__init__.py. If they are unmatched, you are recommended to use one source only to avoid conflict. For example, you can unset SPARK_HOME and use p

In [3]:
config = YoloV1_config
valid_dataset = eval(config.datasets.name)(config, 'valid').get_dataset()
train_dataset = eval(config.datasets.name)(config, 'train').get_dataset()

['hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_valid.tfrecord']
['hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord', 'hdfs://manager:8020/zoo-datasets/voc_train.tfrecord']


In [4]:
def valid_data_creator(config):
    global valid_dataset
    return valid_dataset.batch(config["batch_size"])
def train_data_creator(config):
    global train_dataset
    return train_dataset.batch(config["batch_size"])


In [5]:
net = eval(config.net_name)(config)
model = net.build(config.input_shape)
#print(model.summary())

if config.premodel_path and config.use_pretrained:
    model.load_weights(config.premodel_path, by_name=True, skip_mismatch=True)

# 实例化损失函数
loss = eval(config.loss)(config)

adam = keras.optimizers.Adam(lr=config.lr)
model.compile(optimizer=adam, loss=loss, run_eagerly=True)

In [6]:
def model_creator(config):
    global model
    return model

In [7]:
batch_size = 30
cfg = {
    "batch_size": batch_size
}
model = model_creator(cfg)
train_dataset = train_data_creator(cfg)
valid_dataset = valid_data_creator(cfg)
model.fit(train_dataset, epochs=2, 
          steps_per_epoch=2000 // batch_size,
          validation_data=valid_dataset,
          validation_steps=2000 // batch_size,
          initial_epoch=0)

Epoch 1/2
	 all loss: 61.7627, box_loss: 41.5612, obj_loss: 15.2132, cla_loss: 4.9883
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
55/66 [========================>.....] - ETA: 6:01 - loss: 29.5383	 all loss: 22.7941, box_loss: 17.0664, obj_loss: 2.1523, cla_loss: 3.5754


66/66 [==============================] - ETA: 0s - loss: 28.6077 	 all loss: 38.0763, box_loss: 32.6465, obj_loss: 1.5667, cla_loss: 3.8631
	 all loss: 80.2851, box_loss: 71.4926, obj_loss: 2.8000, cla_loss: 5.9925
	 all loss: 67.3449, box_loss: 59.7155, obj_loss: 2.3000, cla_loss: 5.3294
	 all loss: 45.4262, box_loss: 38.6962, obj_loss: 1.8000, cla_loss: 4.9301
	 all loss: 55.1781, box_loss: 47.2157, obj_loss: 2.1000, cla_loss: 5.8623
	 all loss: 64.8331, box_loss: 58.1031, obj_loss: 2.3333, cla_loss: 4.3967
	 all loss: 54.9562, box_loss: 47.5258, obj_loss: 2.2333, cla_loss: 5.1970
	 all loss: 66.8072, box_loss: 58.3127, obj_loss: 2.3667, cla_loss: 6.1278
	 all loss: 56.7623, box_loss: 49.4679, obj_loss: 2.1000, cla_loss: 5.1945
	 all loss: 68.6442, box_loss: 60.7489, obj_loss: 2.4333, cla_loss: 5.4620
	 all loss: 84.2644, box_loss: 75.1367, obj_loss: 2.6000, cla_loss: 6.5277
	 all loss: 61.4856, box_loss: 54.3213, obj_loss: 2.2333, cla_loss: 4.9310
	 all loss: 74.0022, box_loss: 66.1

66/66 [==============================] - ETA: 0s - loss: 22.9568 	 all loss: 32.6273, box_loss: 27.4201, obj_loss: 1.6000, cla_loss: 3.6071
	 all loss: 47.0177, box_loss: 38.6851, obj_loss: 2.7999, cla_loss: 5.5327
	 all loss: 41.0183, box_loss: 34.0478, obj_loss: 2.2333, cla_loss: 4.7372
	 all loss: 34.5126, box_loss: 29.5966, obj_loss: 1.7000, cla_loss: 3.2160
	 all loss: 36.1757, box_loss: 30.0401, obj_loss: 1.9000, cla_loss: 4.2356
	 all loss: 52.0052, box_loss: 44.1328, obj_loss: 2.6000, cla_loss: 5.2724
	 all loss: 41.6762, box_loss: 33.9426, obj_loss: 2.5333, cla_loss: 5.2003


	 all loss: 43.4282, box_loss: 35.9859, obj_loss: 2.4000, cla_loss: 5.0423
	 all loss: 36.9783, box_loss: 30.8583, obj_loss: 2.0666, cla_loss: 4.0533
	 all loss: 44.3230, box_loss: 36.7578, obj_loss: 2.4333, cla_loss: 5.1319
	 all loss: 37.5919, box_loss: 30.0629, obj_loss: 2.3666, cla_loss: 5.1624
	 all loss: 37.9444, box_loss: 31.4882, obj_loss: 2.2000, cla_loss: 4.2563
	 all loss: 42.3033, box_loss: 35.4222, obj_loss: 2.2333, cla_loss: 4.6478
	 all loss: 40.7715, box_loss: 33.7106, obj_loss: 2.2666, cla_loss: 4.7943
	 all loss: 53.6467, box_loss: 44.4496, obj_loss: 2.9000, cla_loss: 6.2972
	 all loss: 43.6033, box_loss: 35.1110, obj_loss: 2.7333, cla_loss: 5.7590
	 all loss: 38.9938, box_loss: 32.8272, obj_loss: 2.0333, cla_loss: 4.1332
	 all loss: 36.8511, box_loss: 30.9219, obj_loss: 1.9999, cla_loss: 3.9293
	 all loss: 49.7879, box_loss: 42.2234, obj_loss: 2.5666, cla_loss: 4.9979
	 all loss: 33.8461, box_loss: 28.0821, obj_loss: 1.8333, cla_loss: 3.9308
	 all loss: 38.0650, box_

In [8]:
def main(max_epoch):
    batch_size = 10
    config = {
        "batch_size": batch_size
    }
    
    est = Estimator.from_keras(model_creator, config=config, workers_per_node=2)
    
    # callback
    model_checkpoint_callback = cck(filepath="/tmp/yolov1", monitor='val_loss', mode='min',
                                    save_weights_only=False)
    lr_scheduler_callback = LearningRateScheduler(lr_stage=[10, 15, 20], lr_decay=0.1)
    callback_list = [model_checkpoint_callback, lr_scheduler_callback]
    
    history = est.fit(train_data_creator,
                    epochs=max_epoch,
                    steps_per_epoch=20000 // batch_size,
                    validation_data_creator=valid_data_creator,
                    validation_steps=2000 // batch_size,
                    callbacks = callback_list
                     )
    
    est.save("/tmp/yolov1.ckpt")
    est.restore("/tmp/yolov1.ckpt")
    stats = est.evaluate(valid_data_creator, steps=2000 // batch_size)
    print(stats)
    return history

In [9]:
%%time
import time
env={"LD_LIBRARY_PATH": os.environ["LD_LIBRARY_PATH"],
    "HADOOP_HDFS_HOME": os.environ["LD_LIBRARY_PATH"],
    "CLASSPATH":os.environ["CLASSPATH"]}
cluster_mode = "yarn"
if cluster_mode == "local":
    init_orca_context(cluster_mode="local", cores=2, init_ray_on_spark=True)
elif cluster_mode == "yarn":
    init_orca_context(cluster_mode="yarn-client", num_nodes=4, cores=2,
                      init_ray_on_spark=True, memory="32g", driver_memory="2g",hadoop_user_name='hdfs', hadoop_conf="/etc/hadoop/3.0.1.0-187/0/",env=env)
start_time = time.time()
history = main(1)
print("Total: ", time.time() - start_time)

Initializing orca context
Current pyspark location is : /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/pyspark/__init__.py
Initializing SparkContext for yarn-client mode
Start to pack current python env
Packing has been completed: /tmp/tmp1ffrgd27/python_env.tar.gz
pyspark_submit_args is: --master yarn --deploy-mode client --archives /tmp/tmp1ffrgd27/python_env.tar.gz#python_env --driver-cores 4 --driver-memory 2g --num-executors 4 --executor-cores 2 --executor-memory 32g --driver-class-path /root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/zoo/share/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.9.0-SNAPSHOT-jar-with-dependencies.jar:/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/bigdl/share/lib/bigdl-0.12.1-jar-with-dependencies.jar pyspark-shell
JavaGatewayServer has been successfully launched on executors
Start to launch ray on cluster
Start to launch ray driver on local
Executing command: ray start --address 192.168.1.1:48863 --redis-password 123456 

2020-12-22 09:33:22,484	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.



2020-12-22 09:33:22,387	INFO scripts.py:429 -- Using IP address 192.168.1.2 for this node.
2020-12-22 09:33:22,391	INFO resource_spec.py:212 -- Starting Ray with 161.13 GiB memory available for workers and up to 69.07 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-12-22 09:33:22,391	WARNING services.py:1470 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-12-22 09:33:22,413	INFO scripts.py:438 -- 
Started Ray on this node. If you wish to terminate the processes that have been started, run

    ray stop


{'node_ip_address': '192.168.1.2', 'redis_address': '192.168.1.1:48863', 'object_stor

TypeError: can't pickle _thread.RLock objects

Exception ignored in: 'ray._raylet.prepare_args'
Traceback (most recent call last):
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 378, in serialize
    raise e
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 375, in serialize
    value, protocol=5, buffer_callback=writer.buffer_callback)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 617, in dump
    return Pickler.dump(self, obj)
TypeError: can't pickle _thread.RLock objects


...

Exception ignored in: 'ray._raylet.prepare_args'
Traceback (most recent call last):
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 378, in serialize
    raise e
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 375, in serialize
    value, protocol=5, buffer_callback=writer.buffer_callback)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 617, in dump
    return Pickler.dump(self, obj)
TypeError: can't pickle _thread.RLock objects


TypeError: can't pickle _thread.RLock objects

Exception ignored in: 'ray._raylet.prepare_args'
Traceback (most recent call last):
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 378, in serialize
    raise e
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/serialization.py", line 375, in serialize
    value, protocol=5, buffer_callback=writer.buffer_callback)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/root/anaconda3/envs/zoo-tf2.3/lib/python3.6/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 617, in dump
    return Pickler.dump(self, obj)
TypeError: can't pickle _thread.RLock objects


(pid=188804, ip=192.168.1.1) Prepending /hadoop/yarn/local/usercache/hdfs/appcache/application_1608014057528_0031/container_e15_1608014057528_0031_01_000005/python_env/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
(pid=188804, ip=192.168.1.1) Prepending /hadoop/yarn/local/usercache/hdfs/appcache/application_1608014057528_0031/container_e15_1608014057528_0031_01_000005/python_env/lib/python3.6/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path
(pid=188679, ip=192.168.1.1) Prepending /hadoop/yarn/local/usercache/hdfs/appcache/application_1608014057528_0031/container_e15_1608014057528_0031_01_000004/python_env/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
(pid=188679, ip=192.168.1.1) Prepending /hadoop/yarn/local/usercache/hdfs/appcache/application_1608014057528_0031/container_e15_1608014057528_0031_01_000004/python_env/lib/python3.6/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path
(pid=188731, ip=192.

RayTaskError(AssertionError): [36mray::IDLE[39m (pid=188804, ip=192.168.1.1)
  File "python/ray/_raylet.pyx", line 414, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 440, in ray._raylet.execute_task
  File "/hadoop/yarn/local/usercache/hdfs/appcache/application_1608014057528_0031/container_e15_1608014057528_0031_01_000005/python_env/lib/python3.6/site-packages/ray/signature.py", line 139, in recover_args
    "Flattened arguments need to be even-numbered. See `flatten_args`.")
AssertionError: Flattened arguments need to be even-numbered. See `flatten_args`.